# Load GISS-G output from Nick and recompute necessary grid information

To compute transports

In [1]:
import os 
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from xgcm import Grid
from pych.calc import haversine

In [2]:
giss_dir_g = '/workspace/results/giss-euc/ocn_1992-2019_g/'

In [3]:
giss = xr.open_dataset(f'{giss_dir_g}/1992-2019.ug.nc')

for f in ['v','t','s']:
    fld = f if f[-1]!='_' else f[:-1]
    giss[fld] = xr.open_dataarray(f'{giss_dir_g}/1992-2019.{f}g.nc')

In [4]:
giss

<xarray.Dataset>
Dimensions:  (lato: 17, lato2: 16, lono: 288, lono2: 288, record: 336, zoc: 20)
Coordinates:
  * lato     (lato) float32 -3.5 -2.5 -1.5 -0.5 0.5 ... 8.5 9.5 10.5 11.5 12.5
  * lono2    (lono2) float32 -178.75 -177.5 -176.25 ... 177.5 178.75 180.0
  * zoc      (zoc) float32 5.0 16.0 29.0 44.0 61.0 ... 440.0 524.0 625.0 745.0
  * lato2    (lato2) float32 -3.0 -2.0 -1.0 0.0 1.0 ... 8.0 9.0 10.0 11.0 12.0
  * lono     (lono) float32 -179.375 -178.125 -176.875 ... 178.125 179.375
Dimensions without coordinates: record
Data variables:
    u        (record, zoc, lato, lono2) float32 ...
    v        (record, zoc, lato2, lono) float32 ...
    t        (record, zoc, lato, lono) float32 ...
    s        (record, zoc, lato, lono) float32 ...
Attributes:
    xlabel:                    E213f10aF40oQ40 (E213F40oQ32 + all 10 forcings)
    fromto:                    From:  1992  JAN  1,  Hr  0      To:  1992  FE...
    history:                   Wed Mar 18 11:16:24 2020: ncecat 1992-01.ug.nc...
    nco_openmp_thread_number:  1
    NCO:                       4.2.3

In [5]:
time = np.arange('1992-01','2020-01',dtype='datetime64')

In [6]:
assert len(time)==len(giss.record)

### Units

GISS saves U and V in cm/s, swap to m/s

In [7]:
for f in ['u','v']:
    att = giss[f].attrs
    giss[f]=giss[f]/100
    giss[f].attrs=att
    giss[f].attrs['units']='m/s'

In [8]:
giss['time']=xr.DataArray(time,coords=giss.record.coords,dims='record')

In [9]:
giss = giss.swap_dims({'record':'time'})

In [10]:
giss

<xarray.Dataset>
Dimensions:  (lato: 17, lato2: 16, lono: 288, lono2: 288, time: 336, zoc: 20)
Coordinates:
  * lato     (lato) float32 -3.5 -2.5 -1.5 -0.5 0.5 ... 8.5 9.5 10.5 11.5 12.5
  * lono2    (lono2) float32 -178.75 -177.5 -176.25 ... 177.5 178.75 180.0
  * zoc      (zoc) float32 5.0 16.0 29.0 44.0 61.0 ... 440.0 524.0 625.0 745.0
  * lato2    (lato2) float32 -3.0 -2.0 -1.0 0.0 1.0 ... 8.0 9.0 10.0 11.0 12.0
  * lono     (lono) float32 -179.375 -178.125 -176.875 ... 178.125 179.375
  * time     (time) datetime64[ns] 1992-01-01 1992-02-01 ... 2019-12-01
Data variables:
    u        (time, zoc, lato, lono2) float32 -0.105167694 ... 0.0010093153
    v        (time, zoc, lato2, lono) float32 -0.03161063 ... 0.0020525772
    t        (time, zoc, lato, lono) float32 ...
    s        (time, zoc, lato, lono) float32 ...
Attributes:
    xlabel:                    E213f10aF40oQ40 (E213F40oQ32 + all 10 forcings)
    fromto:                    From:  1992  JAN  1,  Hr  0      To:  1992  FE...
    history:                   Wed Mar 18 11:16:24 2020: ncecat 1992-01.ug.nc...
    nco_openmp_thread_number:  1
    NCO:                       4.2.3

### Rename some coordinates

Just use MITgcm style to make it easy for me

the atmospheric variables sit at the intersection of four grid cells, hence why the XG/YG points align with the atm variable lon/lat

In [11]:
giss = giss.rename({'lono2':'XG','lono':'XC','lato2':'YG','lato':'YC','zoc':'Z'})

In [12]:
giss.data_vars

Data variables:
    u        (time, Z, YC, XG) float32 -0.105167694 -0.10802157 ... 0.0010093153
    v        (time, Z, YG, XC) float32 -0.03161063 -0.033070985 ... 0.0020525772
    t        (time, Z, YC, XC) float32 ...
    s        (time, Z, YC, XC) float32 ...

In [13]:
giss.YC

<xarray.DataArray 'YC' (YC: 17)>
array([-3.5, -2.5, -1.5, -0.5,  0.5,  1.5,  2.5,  3.5,  4.5,  5.5,  6.5,  7.5,
        8.5,  9.5, 10.5, 11.5, 12.5], dtype=float32)
Coordinates:
  * YC       (YC) float32 -3.5 -2.5 -1.5 -0.5 0.5 1.5 ... 8.5 9.5 10.5 11.5 12.5
Attributes:
    units:    degrees_north

In [14]:
giss.YG

<xarray.DataArray 'YG' (YG: 16)>
array([-3., -2., -1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.,
       11., 12.], dtype=float32)
Coordinates:
  * YG       (YG) float32 -3.0 -2.0 -1.0 0.0 1.0 2.0 ... 8.0 9.0 10.0 11.0 12.0
Attributes:
    units:    degrees_north

### Now there is one extra eastward velocity point which is giving some trouble

Weird, not sure why that is there...

In [15]:
giss = giss.drop_sel(YC=-3.5)

### Some meta data for xgcm

and another vertical coordinate...

In [16]:
giss.XC.attrs['axis'] = 'X'
giss.XG.attrs['axis'] = 'X'

giss.YC.attrs['axis'] = 'Y'
giss.YG.attrs['axis'] = 'Y'

giss.XG.attrs['c_grid_axis_shift']=-0.5
giss.YG.attrs['c_grid_axis_shift']=-0.5

In [40]:
giss.Z.shift(Z=-1)

<xarray.DataArray 'Z' (Z: 20)>
array([ 16.,  29.,  44.,  61.,  80., 100., 120., 140., 160., 181., 205., 234.,
       270., 315., 371., 440., 524., 625., 745.,  nan], dtype=float32)
Coordinates:
  * Z        (Z) float32 5.0 16.0 29.0 44.0 61.0 ... 440.0 524.0 625.0 745.0
Attributes:
    units:     m
    positive:  down
    axis:      Z

In [18]:
np.concatenate(np.array([[0],(giss.Z+giss.Z.shift(Z=-1)[:-1])/2]))

array([  0. ,  10.5,  22.5,  36.5,  52.5,  70.5,  90. , 110. , 130. ,
       150. , 170.5, 193. , 219.5, 252. , 292.5, 343. , 405.5, 482. ,
       574.5, 685. ])

In [19]:
giss['Zl'] =np.concatenate(np.array([[0],(giss.Z+giss.Z.shift(Z=-1)[:-1])/2]))

In [20]:
giss.Zl

<xarray.DataArray 'Zl' (Zl: 20)>
array([  0. ,  10.5,  22.5,  36.5,  52.5,  70.5,  90. , 110. , 130. , 150. ,
       170.5, 193. , 219.5, 252. , 292.5, 343. , 405.5, 482. , 574.5, 685. ])
Coordinates:
  * Zl       (Zl) float64 0.0 10.5 22.5 36.5 52.5 ... 405.5 482.0 574.5 685.0

Have to fake the other "side" of the Z axis

In [21]:
dz = giss.Z[-1]-giss.Zl[-1]
giss['Zu'] = np.concatenate([giss.Zl[1:],[(giss.Z[-1]+dz)]])

In [22]:
giss['Zp1'] = np.concatenate([[giss.Zl[0]],giss.Zu])

In [23]:
giss.Z.attrs['axis'] = 'Z'
giss.Zl.attrs['axis'] = 'Z'
giss.Zu.attrs['axis'] = 'Z'
giss.Zp1.attrs['axis'] = 'Z'

giss.XG.attrs['c_grid_axis_shift']=-0.5
giss.YG.attrs['c_grid_axis_shift']=-0.5
giss.Zl.attrs['c_grid_axis_shift']=-0.5
giss.Zu.attrs['c_grid_axis_shift']= 0.5
giss.Zp1.attrs['c_grid_axis_shift']=0.5

In [24]:
giss

<xarray.Dataset>
Dimensions:  (XC: 288, XG: 288, YC: 16, YG: 16, Z: 20, Zl: 20, Zp1: 21, Zu: 20, time: 336)
Coordinates:
  * YC       (YC) float32 -2.5 -1.5 -0.5 0.5 1.5 2.5 ... 8.5 9.5 10.5 11.5 12.5
  * XG       (XG) float32 -178.75 -177.5 -176.25 -175.0 ... 177.5 178.75 180.0
  * Z        (Z) float32 5.0 16.0 29.0 44.0 61.0 ... 440.0 524.0 625.0 745.0
  * YG       (YG) float32 -3.0 -2.0 -1.0 0.0 1.0 2.0 ... 8.0 9.0 10.0 11.0 12.0
  * XC       (XC) float32 -179.375 -178.125 -176.875 ... 176.875 178.125 179.375
  * time     (time) datetime64[ns] 1992-01-01 1992-02-01 ... 2019-12-01
  * Zl       (Zl) float64 0.0 10.5 22.5 36.5 52.5 ... 405.5 482.0 574.5 685.0
  * Zu       (Zu) float64 10.5 22.5 36.5 52.5 70.5 ... 482.0 574.5 685.0 805.0
  * Zp1      (Zp1) float64 0.0 10.5 22.5 36.5 52.5 ... 482.0 574.5 685.0 805.0
Data variables:
    u        (time, Z, YC, XG) float32 -0.18397406 -0.18022417 ... 0.0010093153
    v        (time, Z, YG, XC) float32 -0.03161063 -0.033070985 ... 0.0020525772
    t        (time, Z, YC, XC) float32 ...
    s        (time, Z, YC, XC) float32 ...
Attributes:
    xlabel:                    E213f10aF40oQ40 (E213F40oQ32 + all 10 forcings)
    fromto:                    From:  1992  JAN  1,  Hr  0      To:  1992  FE...
    history:                   Wed Mar 18 11:16:24 2020: ncecat 1992-01.ug.nc...
    nco_openmp_thread_number:  1
    NCO:                       4.2.3

### Finally, get differential elements for computing transports

dx, dy...

In [25]:
xg,yg = np.meshgrid(giss.XG,giss.YG)

In [26]:
dxG = haversine(xg,yg,np.roll(xg,-1,axis=1),np.roll(yg,-1,axis=1))*1000

Since latitude not periodic, have to add on the boundary condition

In [27]:
xg_rolled = np.roll(xg,-1,axis=0)
yg_rolled = np.roll(yg,-1,axis=0)

yg_rolled[-1,:] = 13.*np.ones_like(xg_rolled[0,:])

In [28]:
dyG = haversine(xg_rolled,yg_rolled,xg,yg) * 1000

In [29]:
xc,yc = np.meshgrid(giss.XC,giss.YC)

In [30]:
dxC = haversine(xc,yc,np.roll(xc,-1,axis=1),np.roll(yc,-1,axis=1))*1000

In [31]:
xc_rolled = np.roll(xc,-1,axis=0)
yc_rolled = np.roll(yc,-1,axis=0)

yc_rolled[-1,:]=13.5*np.ones_like(yc_rolled[0,:])

In [32]:
dyC = haversine(xc_rolled,yc_rolled,xc,yc)*1000

In [33]:
giss['dxG'] = xr.DataArray(dxG,coords={'YG':giss.YG,'XC':giss.XC},dims=('YG','XC'),
                          attrs={'units':'m'})
giss['dyG'] = xr.DataArray(dyG,coords={'YC':giss.YC,'XG':giss.XG},dims=('YC','XG'),
                          attrs={'units':'m'})
giss['dxC'] = xr.DataArray(dxC,coords={'YC':giss.YC,'XG':giss.XG},dims=('YC','XG'),
                          attrs={'units':'m'})
giss['dyC'] = xr.DataArray(dyC,coords={'YG':giss.YG,'XC':giss.XC},dims=('YG','XC'),
                          attrs={'units':'m'})

In [34]:
for f in ['dxG','dxC','dyC','dyG']:
    giss=giss.set_coords(f)

### Now make a grid object

In [35]:
grid = Grid(giss,periodic='X')

In [36]:
grid

<xgcm.Grid>
Z Axis (not periodic):
  * center   Z --> left
  * left     Zl --> center
  * outer    Zp1 --> center
  * right    Zu --> center
Y Axis (not periodic):
  * center   YC --> left
  * left     YG --> center
X Axis (periodic):
  * center   XC --> left
  * left     XG --> center

In [37]:
giss['drF'] = grid.diff(giss.Zp1,'Z',to='center')

## Save this for later

In [38]:
giss.to_netcdf('/workspace/results/giss-euc/giss_g.nc')